## Using a pipeline

In [1]:
from transformers import pipeline
nlp = pipeline("ner")
sequence = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very close to the Manhattan Bridge which is visible from the window."

In [2]:
print(nlp(sequence))

[{'word': 'Hu', 'score': 0.999578595161438, 'entity': 'I-ORG', 'index': 1}, {'word': '##gging', 'score': 0.9909763932228088, 'entity': 'I-ORG', 'index': 2}, {'word': 'Face', 'score': 0.9982224702835083, 'entity': 'I-ORG', 'index': 3}, {'word': 'Inc', 'score': 0.9994880557060242, 'entity': 'I-ORG', 'index': 4}, {'word': 'New', 'score': 0.9994345307350159, 'entity': 'I-LOC', 'index': 11}, {'word': 'York', 'score': 0.9993196129798889, 'entity': 'I-LOC', 'index': 12}, {'word': 'City', 'score': 0.9993793964385986, 'entity': 'I-LOC', 'index': 13}, {'word': 'D', 'score': 0.9862582683563232, 'entity': 'I-LOC', 'index': 19}, {'word': '##UM', 'score': 0.9514270424842834, 'entity': 'I-LOC', 'index': 20}, {'word': '##BO', 'score': 0.9336591362953186, 'entity': 'I-LOC', 'index': 21}, {'word': 'Manhattan', 'score': 0.9761654138565063, 'entity': 'I-LOC', 'index': 28}, {'word': 'Bridge', 'score': 0.9914628863334656, 'entity': 'I-LOC', 'index': 29}]


## Using a model and a tokenizer

In [3]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

In [4]:
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english", return_dict=True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [5]:
label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation right after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]

In [6]:
sequence = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very" \
           "close to the Manhattan Bridge."

# Bit of a hack to get the tokens with the special tokens
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs).logits
predictions = torch.argmax(outputs, dim=2)

In [7]:
print(predictions)

tensor([[0, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 8, 8, 8, 0, 0, 0, 0, 0, 8, 8, 8, 0, 0,
         0, 0, 0, 0, 0, 8, 8, 0, 0]])


In [8]:
print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())])

[('[CLS]', 'O'), ('Hu', 'I-ORG'), ('##gging', 'I-ORG'), ('Face', 'I-ORG'), ('Inc', 'I-ORG'), ('.', 'O'), ('is', 'O'), ('a', 'O'), ('company', 'O'), ('based', 'O'), ('in', 'O'), ('New', 'I-LOC'), ('York', 'I-LOC'), ('City', 'I-LOC'), ('.', 'O'), ('Its', 'O'), ('headquarters', 'O'), ('are', 'O'), ('in', 'O'), ('D', 'I-LOC'), ('##UM', 'I-LOC'), ('##BO', 'I-LOC'), (',', 'O'), ('therefore', 'O'), ('very', 'O'), ('##c', 'O'), ('##lose', 'O'), ('to', 'O'), ('the', 'O'), ('Manhattan', 'I-LOC'), ('Bridge', 'I-LOC'), ('.', 'O'), ('[SEP]', 'O')]
